In [ ]:
import json

# Load JSON data
input_file = "squad_formatted_dataset_multi_shot.json"
output_file = "Modelfile"  # Added .txt extension for clarity

try:
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
except json.JSONDecodeError as e:
    print(f"Error loading JSON file: {e}")
    exit()

# Start creating the model file content
formatted_model_instructions = """
FROM llama3.2

PARAMETER temperature 0.5
PARAMETER num_ctx 4096
# set the system prompt
SYSTEM \"\"\"
You are programmed to simulate a native Arabic speaker. Your responses must be exclusively in Arabic.

Your operation involves a three-step process for every Arabic question:
1. Silently translate the question from Arabic to English.
2. Formulate the response in English based on the knowledge provided in the dataset.
3. Translate your English response back into Arabic, and provide this as your reply.

Do not provide the English translation of the question or the English version of the answer in your response.

Here are examples derived from the dataset to illustrate this process:
"""

# Extract examples from the JSON data
examples = []
for entry in data['data']:
    for paragraph in entry['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            for answer in qa['answers']:
                answer_text = answer['text']
                if answer_text.strip():  # Avoid empty answers
                    examples.append((question, answer_text))

# Add examples to the model content
for i, (question, answer) in enumerate(examples):
    formatted_model_instructions += f"""
-- Example {i + 1}:

USER:
    {question}

(Translated-question: Translation not shown here for brevity.)

(Answer in English: Translation not shown here for brevity.)

ASSISTANT:
    {answer.strip()}
"""

# Close the SYSTEM section
formatted_model_instructions += """
Remember, only provide the Arabic translation in your response. Use examples from the dataset to ensure consistency with user expectations.
\"\"\"
"""

# Save to output file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(formatted_model_instructions)

print(f"Model file generated and saved to {output_file}")
